In [ ]:
!pip install transformers

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

# Encode

In [ ]:
from transformers import BertTokenizer, BertForQuestionAnswering

# dataloaders
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [ ]:
!pip install datasets
from datasets import load_dataset

In [ ]:
train_dataset = load_dataset("squad", split="train")
validation_dataset = load_dataset("squad", split="validation")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(train_dataset)

In [ ]:
### PreProcesse Data

In [ ]:
def find_end(example):

    if (len(example['answers']['text']) != 0):
        context = example['context']
        text = example['answers']['text'][0]
        start_idx = example['answers']['answer_start'][0]

        end_idx = start_idx + len(text)

        temp = example['answers'] # to change the value
        temp['answer_end']=end_idx
        temp['answer_start'] = start_idx # [num]->num
        temp['text'] = text # ['text']->text

    else:
        temp = example['answers']
        temp['answer_end'] = 0 # []->0
        temp['answer_start'] = 0 # []->0
        temp['text'] = "" # []->""

    return example

train_dataset = train_dataset.map(find_end)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

In [ ]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

tokenized_train = tokenizer(train_dataset['context'], train_dataset['question'], truncation=True, padding=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
def find_token_indexes(tokenized, dataset):
    start_token_list = []
    end_token_list = []
    answers = dataset['answers']
    for i in range(len(answers)):
        if (answers[i]['text'] != ''):
            start_token = tokenized.char_to_token(i, answers[i]['answer_start'])
            end_token = tokenized.char_to_token(i, answers[i]['answer_end'] - 1)

            # if start token is None, the answer passage has been truncated
            if start_token is None:
                start_token = tokenizer.model_max_length
            if end_token is None:
                end_token = tokenizer.model_max_length
        else:
            start_token = 0
            end_token = 0

        start_token_list.append(start_token)
        end_token_list.append(end_token)

    return start_token_list, start_token_list

s, e = find_token_indexes(tokenized_train, train_dataset)
train_dataset = train_dataset.add_column("start_position", s)
train_dataset = train_dataset.add_column("end_position", e)

# PyTorch FineTuning


In [ ]:
import torch

In [ ]:
batch_size = 8
train_data = TensorDataset(torch.tensor(tokenized_train['input_ids'], dtype=torch.int64),
                           torch.tensor(tokenized_train['token_type_ids'], dtype=torch.int64),
                           torch.tensor(tokenized_train['attention_mask'], dtype=torch.float),
                           torch.tensor(train_dataset['start_position'], dtype=torch.int64),
                           torch.tensor(train_dataset['start_position'], dtype=torch.int64))

train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [ ]:
tokenized_validation = tokenizer(validation_dataset['context'], validation_dataset['question'], truncation=True, padding=True, return_offsets_mapping=True)

In [ ]:
batch_size = 8
val_data = TensorDataset(torch.tensor(tokenized_validation['input_ids'], dtype=torch.int64),
                        torch.tensor(tokenized_validation['token_type_ids'], dtype=torch.int64),
                        torch.tensor(tokenized_validation['attention_mask'], dtype=torch.float))
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

In [ ]:
### Fine tuning

In [ ]:
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import torch.optim as optim

In [ ]:
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')
epochs = 3
model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-5)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from tqdm import tqdm

for epoch in range(epochs):
    epoch_loss = []
    validation_loss = []

    total_loss = 0
    model.train()

    count=-1
    progress_bar = tqdm(train_dataloader, leave=True, position=0)
    progress_bar.set_description(f"Epoch {epoch+1}")
    for batch in progress_bar:
        count+=1
        input_ids, segment_ids, mask, start, end  = tuple(t.to(device) for t in batch)

        model.zero_grad()
        loss, start_logits, end_logits = model(input_ids = input_ids,
                                                token_type_ids = segment_ids,
                                                attention_mask = mask,
                                                start_positions = start,
                                                end_positions = end,
                                                return_dict = False)

        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        if (count % 20 == 0 and count != 0):
            avg = total_loss/count
            progress_bar.set_postfix(Loss=avg)



Epoch 3: 100%|██████████| 10950/10950 [34:09<00:00,  5.34it/s, Loss=0.645]


# Save Model


In [ ]:
model_path = 'models/bert1-squad'
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('models/bert-squad/tokenizer_config.json',
 'models/bert-squad/special_tokens_map.json',
 'models/bert-squad/vocab.txt',
 'models/bert-squad/added_tokens.json',
 'models/bert-squad/tokenizer.json')

In [ ]:
model.push_to_hub("bert1-squad")
tokenizer.push_to_hub("bert1-squad")

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Dofla/bert-squad/commit/5877851420419221436444d86497bc18064b952a', commit_message='Upload tokenizer', commit_description='', oid='5877851420419221436444d86497bc18064b952a', pr_url=None, pr_revision=None, pr_num=None)